In [ ]:
#### kobert 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-r0vgb_3_/kobert-tokenizer_e2ebf710c85b4ed09ff025e43e3e94cd
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-r0vgb_3_/kobert-tokenizer_e2ebf710c85b4ed09ff025e43e3e94cd
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=aaa616da5a68fbae9e0b74c9e13a5e9cecefcaabf4dd4ee29b112e030b89c7ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-ds9037mr/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
### requirements
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661672 sha256=c943ee4937a9f85a0f3f4d99381f0e0ec7c5240e7835af8f63517feb9cc4e05d
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer

device = torch.device("cuda")

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
import pandas as pd
dataset_train_csv =  pd.read_csv('/content/drive/MyDrive/KHUThon/한상진팀/비밀의 폴더/Data/final.csv')

In [ ]:
dataset_train_csv

,text,label
0,애덜앙 나 너무 배불러서 배 아파,1
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,얼마나 먹었다고 배가 아프데,1
3,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1
4,하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1
...,...,...
1327782,나두 피부샵 가서 전신마사지 받고 싶어,16
1327783,요즘은 조금씩 보톡스 잘 맞더라,16
1327784,지금부터라도 관리해야 오십되어서 후회 안 하겠지,16
1327785,얼마전에 개원한 곳에서 보톡스 세일 전단지 봤어,16


In [ ]:
dataset_train_csv['label'] -= 1  # 1~20으로 하니깐 unique_by_key: failed to synchronize: cudaErrorAssert: device-side assert triggered 라는 오류가 loss.backward에서 계속 발생

In [ ]:
dataset_train_csv.loc[dataset_train_csv['label'] == 0, 'label'] = -1
dataset_train_csv.loc[dataset_train_csv['label'] == 14, 'label'] = 0
dataset_train_csv.loc[dataset_train_csv['label'] == -1, 'label'] = 14

In [ ]:
dataset_train_csv

,text,label
0,애덜앙 나 너무 배불러서 배 아파,14
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,14
2,얼마나 먹었다고 배가 아프데,14
3,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,14
4,하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,14
...,...,...
1327782,나두 피부샵 가서 전신마사지 받고 싶어,15
1327783,요즘은 조금씩 보톡스 잘 맞더라,15
1327784,지금부터라도 관리해야 오십되어서 후회 안 하겠지,15
1327785,얼마전에 개원한 곳에서 보톡스 세일 전단지 봤어,15


In [ ]:
dataset_train = []
for q, label in zip(dataset_train_csv['text'], dataset_train_csv['label']):
    data = []
    data.append(q)
    data.append(str(label))

    dataset_train.append(data)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# hyperparameter
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,   ##모델의 마지막층의 class를 지정해줘야 한다. (카테고리개수)##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss = loss.to(device)
        loss.backward()
        torch.cuda.synchronize()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

<ipython-input-37-ff95e890c7f2>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/20747 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.901762008666992 train acc 0.203125
epoch 1 batch id 201 loss 2.988175868988037 train acc 0.05977922885572139
epoch 1 batch id 401 loss 3.065916061401367 train acc 0.06959164588528678
epoch 1 batch id 601 loss 2.9832630157470703 train acc 0.09213810316139767
epoch 1 batch id 801 loss 2.8740029335021973 train acc 0.13048142946317104
epoch 1 batch id 1001 loss 2.4094464778900146 train acc 0.1742007992007992
epoch 1 batch id 1201 loss 2.1415913105010986 train acc 0.21718880099916735
epoch 1 batch id 1401 loss 2.2895119190216064 train acc 0.256134011420414
epoch 1 batch id 1601 loss 1.8877209424972534 train acc 0.27998126171143034
epoch 1 batch id 1801 loss 2.2211456298828125 train acc 0.3030868267629095
epoch 1 batch id 2001 loss 1.5461293458938599 train acc 0.32502498750624687
epoch 1 batch id 2201 loss 1.4862172603607178 train acc 0.3409174806905952
epoch 1 batch id 2401 loss 0.9061752557754517 train acc 0.35567211578508956
epoch 1 batch id 2601 loss 2.419323682

  0%|          | 0/20747 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 6.726292133331299 train acc 0.046875
epoch 2 batch id 201 loss 2.0624725818634033 train acc 0.4304259950248756
epoch 2 batch id 401 loss 2.2093818187713623 train acc 0.4647365960099751
epoch 2 batch id 601 loss 1.8944276571273804 train acc 0.47709546589018303
epoch 2 batch id 801 loss 2.1830077171325684 train acc 0.4842189450686642
epoch 2 batch id 1001 loss 1.7286359071731567 train acc 0.4872627372627373
epoch 2 batch id 1201 loss 1.3809900283813477 train acc 0.49275343463780186
epoch 2 batch id 1401 loss 1.590627670288086 train acc 0.4994312098501071
epoch 2 batch id 1601 loss 1.7935281991958618 train acc 0.4969062304809494
epoch 2 batch id 1801 loss 1.7109313011169434 train acc 0.499731052193226
epoch 2 batch id 2001 loss 1.3758647441864014 train acc 0.5037481259370314
epoch 2 batch id 2201 loss 1.3958081007003784 train acc 0.5046214788732394
epoch 2 batch id 2401 loss 0.6012126207351685 train acc 0.5068916597251145
epoch 2 batch id 2601 loss 2.20778036117553

...

epoch 3 batch id 1 loss 6.670163631439209 train acc 0.0625
epoch 3 batch id 201 loss 1.8853976726531982 train acc 0.4731032338308458
epoch 3 batch id 401 loss 2.001929998397827 train acc 0.5040134039900249
epoch 3 batch id 601 loss 1.6922262907028198 train acc 0.515676996672213
epoch 3 batch id 801 loss 2.098954677581787 train acc 0.5232716916354557
epoch 3 batch id 1001 loss 1.4128280878067017 train acc 0.5258959790209791
epoch 3 batch id 1201 loss 1.2478148937225342 train acc 0.5317703996669442
epoch 3 batch id 1401 loss 1.1987793445587158 train acc 0.5383319950035689
epoch 3 batch id 1601 loss 1.578690528869629 train acc 0.5354661149281699
epoch 3 batch id 1801 loss 1.5564250946044922 train acc 0.5382599944475291
epoch 3 batch id 2001 loss 1.189481496810913 train acc 0.5426036981509246
epoch 3 batch id 2201 loss 1.2323321104049683 train acc 0.543275783734666
epoch 3 batch id 2401 loss 0.443590372800827 train acc 0.5455929820907955
epoch 3 batch id 2601 loss 1.7854098081588745 train 

  0%|          | 0/20747 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 4.582079887390137 train acc 0.28125
epoch 4 batch id 201 loss 1.8287076950073242 train acc 0.5067630597014925
epoch 4 batch id 401 loss 2.0737974643707275 train acc 0.5406405860349127
epoch 4 batch id 601 loss 1.4317841529846191 train acc 0.5534785773710482
epoch 4 batch id 801 loss 1.8563148975372314 train acc 0.5608809300873908
epoch 4 batch id 1001 loss 1.270248293876648 train acc 0.563858016983017
epoch 4 batch id 1201 loss 1.1348905563354492 train acc 0.5693302456286428
epoch 4 batch id 1401 loss 1.062870979309082 train acc 0.5757829229122056
epoch 4 batch id 1601 loss 1.4491841793060303 train acc 0.5726694253591506
epoch 4 batch id 1801 loss 1.4067480564117432 train acc 0.5758346057745697
epoch 4 batch id 2001 loss 1.095467209815979 train acc 0.5801083833083458
epoch 4 batch id 2201 loss 0.9515215754508972 train acc 0.5809717174011813
epoch 4 batch id 2401 loss 0.3609607219696045 train acc 0.5830643481882549
epoch 4 batch id 2601 loss 1.8090054988861084 tr

  0%|          | 0/20747 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 2.837735891342163 train acc 0.453125
epoch 5 batch id 201 loss 1.415095329284668 train acc 0.5279850746268657
epoch 5 batch id 401 loss 2.1646831035614014 train acc 0.5621882793017456
epoch 5 batch id 601 loss 1.4514483213424683 train acc 0.5776570299500832
epoch 5 batch id 801 loss 1.8048118352890015 train acc 0.5849133895131086
epoch 5 batch id 1001 loss 1.0661414861679077 train acc 0.5888486513486514
epoch 5 batch id 1201 loss 0.9527506828308105 train acc 0.5952461490424646
epoch 5 batch id 1401 loss 0.9792064428329468 train acc 0.6025272127052106
epoch 5 batch id 1601 loss 1.2580533027648926 train acc 0.5997813866333541
epoch 5 batch id 1801 loss 1.3383320569992065 train acc 0.6033540394225431
epoch 5 batch id 2001 loss 0.9110114574432373 train acc 0.6075555972013993
epoch 5 batch id 2201 loss 0.8334746956825256 train acc 0.6087928782371649
epoch 5 batch id 2401 loss 0.26681843400001526 train acc 0.6105268638067471
epoch 5 batch id 2601 loss 1.68663108348846

In [ ]:
torch.save(model.state_dict(), #"Kobert_epoch5_plus.pt")